In [67]:
from sagemaker.tensorflow.model import TensorFlowModel
from sagemaker.session import Session
import sagemaker
import sys,os
import tarfile

In [61]:
folder_path = "Hot-DOG"
output_data_path = 'data.tar.gz'
role = "arn:aws:iam::787258559497:role/sagemaker-test"
DATASET_NAME = "Hot-DOG"
#sagemaker_session = Session()

In [62]:
base_dir='/tmp'

%env BASE_DIR=$base_dir
%env S3_DATA_BUCKET_NAME = $data_bucket_name
%env DATASET_NAME = $dataset_name

suffix='/mxnet/tools/im2rec.py'
im2rec = list(filter( (lambda x: os.path.isfile(x + suffix )), sys.path))[0] + suffix
%env IM2REC=$im2rec

env: BASE_DIR=/tmp
env: S3_DATA_BUCKET_NAME=$data_bucket_name
env: DATASET_NAME=data
env: IM2REC=/Users/mashex/anaconda3/lib/python3.7/site-packages/mxnet/tools/im2rec.py


In [63]:
%%bash
cd "Hot-DOG"
ls
rm *.rec
rm *.lst
echo "Creating LST files"
python $IM2REC --list --recursive --pass-through --test-ratio=0.3 --train-ratio=0.7 $DATASET_NAME $DATASET_NAME > ${DATASET_NAME}_classes
echo "Label classes:"
cat ${DATASET_NAME}_classes

# Then we create RecordIO files from the LST files
echo "Creating RecordIO files"
python $IM2REC --num-thread=4 ${DATASET_NAME}_train.lst $DATASET_NAME
python $IM2REC --num-thread=4 ${DATASET_NAME}_test.lst $DATASET_NAME
ls -lh *.rec

$dataset_name_classes
$dataset_name_test.idx
$dataset_name_test.lst
$dataset_name_test.rec
$dataset_name_train.idx
$dataset_name_train.lst
$dataset_name_train.rec
hot_dog
not_hot_dog
Creating LST files
Label classes:
Creating RecordIO files
Creating .rec file from /Users/mashex/Downloads/GitHub/Hackathon-Project-Innovaccer/src/Hot-DOG/data_train.lst in /Users/mashex/Downloads/GitHub/Hackathon-Project-Innovaccer/src/Hot-DOG
Creating .rec file from /Users/mashex/Downloads/GitHub/Hackathon-Project-Innovaccer/src/Hot-DOG/data_test.lst in /Users/mashex/Downloads/GitHub/Hackathon-Project-Innovaccer/src/Hot-DOG
-rw-r--r--  1 mashex  staff     0B Dec 14 12:46 data_test.rec
-rw-r--r--  1 mashex  staff     0B Dec 14 12:46 data_train.rec


In [52]:
bucket = sagemaker_session.default_bucket()
dataset_name = "data"

In [53]:
!aws s3 rm s3://{bucket}/{dataset_name}/train --recursive
!aws s3 rm s3://{bucket}/{dataset_name}/validation --recursive

In [65]:
s3train_path = 's3://{}/{}/train/'.format(bucket, dataset_name)
s3validation_path = 's3://{}/{}/validation/'.format(bucket, dataset_name)
!aws s3 cp {folder_path}/data_train.rec $s3train_path
!aws s3 cp {folder_path}/data_test.rec $s3validation_path

upload: Hot-DOG/data_train.rec to s3://sagemaker-us-west-2-787258559497/data/train/data_train.rec
upload: Hot-DOG/data_test.rec to s3://sagemaker-us-west-2-787258559497/data/validation/data_test.rec


In [68]:
train_data = sagemaker.session.s3_input(
    s3train_path, 
    distribution='FullyReplicated', 
    content_type='application/x-recordio', 
    s3_data_type='S3Prefix'
)

validation_data = sagemaker.session.s3_input(
    s3validation_path, 
    distribution='FullyReplicated', 
    content_type='application/x-recordio', 
    s3_data_type='S3Prefix'
)

data_channels = {'train': train_data, 'validation': validation_data}

In [ ]:
s3_output_location = 's3://{}/{}/output'.format(bucket, dataset_name)

image_classifier = sagemaker.estimator.Estimator(
    training_image,
    role, 
    train_instance_count=1, 
    train_instance_type='ml.p3.2xlarge',
    output_path=s3_output_location,
    sagemaker_session=sess
)

In [10]:

def tardir(path, tar_name):
    with tarfile.open(tar_name, "w:gz") as tar_handle:
        for root, dirs, files in os.walk(path):
            for file in files:
                tar_handle.add(os.path.join(root, file))
tardir(folder_path, output_data_path)

In [16]:
model_data = sagemaker_session.upload_data(path=output_data_path, key_prefix='data')

In [17]:
model_data

's3://sagemaker-us-west-2-787258559497/data/data.tar.gz'

In [26]:
import sys
suffix='/mxnet/tools/im2rec.py'
list(filter( (lambda x: os.path.isfile(x + suffix )), sys.path))[0] + suffix

'/Users/mashex/anaconda3/lib/python3.7/site-packages/mxnet/tools/im2rec.py'